In [1]:
# For Pyomo
from __future__ import division
from itertools import product
import shutil
import sys
import os.path
import numpy as np
import time
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition

In [2]:
# if not shutil.which("pyomo"):
#     get_ipython().system('pip install -q pyomo')
#     assert(shutil.which("pyomo"))

# if not (shutil.which("cbc") or os.path.isfile("cbc")):
#     if "google.colab" in sys.modules:
#         get_ipython().system('apt-get install -y -qq coinor-cbc')
#     else:
#         try:
#             get_ipython().system('conda install -c conda-forge coincbc ')
#         except:
#             pass

# assert(shutil.which("cbc") or os.path.isfile("cbc"))

In [3]:
# ,,,,,,,,,,,,,, Обнуление тех значений, между которыми нет каналов ,,,,,,,,,,,,,,

def zero_nconnect(cnstr, m, Bandwidth):
    for (k, i, j) in product(m.K, m.V, m.V):
        if Bandwidth.at[i, j] == 0:
            for tm in m.M:
                cnstr.add(m.q[i, j, k, tm] == 0)

In [4]:
# # ,,,,,,,,,,,,,, Запрещаем учитывать время после t_k для каждого контракта ,,,,,,,

# def only_selftime(cnstr, m, k, t):
#     for i, j in product(m.V, m.V):
#         cnstr.add(m.q[i, j, k, t] == 0)

In [5]:
# # ,,,,,,,,,,,,,, Условие 0 ,,,,,,,,,,,,,,

# def eval_cnstr0(m, min_t, C, Bandwidth, BackFlows):
#     for tm in m.M:
#         sum_v = 0
#         for i, j in product(m.V, m.V):
#             if Bandwidth.at[i, j] != 0:
#                 sum_v += Bandwidth.at[i, j] - BackFlows.at[i, j]
#         if not (sum_v >= sum(K[k]['volume'] for k in m.K) / min_t):
#             return False
#     return True

In [6]:
# ,,,,,,,,,,,,,, Условие 1 ,,,,,,,,,,,,,,

def eval_cnstr1(cnstr, m, I, Bandwidth, BackFlows):
    for tm in m.M:
        for i in m.V:
            for j in m.V:
                if Bandwidth.at[i, j] != 0:
                    if I[tm]['Ctm'] != set():
                        cnstr.add(sum(m.q[i, j, k, tm] for k in I[tm]['Ctm']) <= Bandwidth.at[i, j] - BackFlows.at[i, j])

In [7]:
# ,,,,,,,,,,,,,, Условие 2 ,,,,,,,,,,,,,,

def eval_cnstr2(cnstr, m, C, I):
    for k in m.K:
        a_k = C[k]['A']
        s = 0
        for tm in m.M:
            if k in I[tm]['Ctm']:
                for j in m.V:
                    s += m.q[a_k, j, k, tm] * I[tm]['em']
        cnstr.add(s == C[k]['volume'])

In [8]:
# ,,,,,,,,,,,,,, Условие 3 ,,,,,,,,,,,,,,

def eval_cnstr3(cnstr, m, C, I):
    for k in m.K:
        a_k = C[k]['A']
        s = 0
        for tm in m.M:
            if k in I[tm]['Ctm']:
                for j in m.V:
                    s += m.q[j, a_k, k, tm] * I[tm]['em']
        cnstr.add(s == 0)

In [9]:
# ,,,,,,,,,,,,,, Условие 4 ,,,,,,,,,,,,,,

def eval_cnstr4(cnstr, m, C, I):
    for k in m.K:
        b_k = C[k]['B']
        s = 0
        for tm in m.M:
            if k in I[tm]['Ctm']:
                for j in m.V:
                    s += m.q[j, b_k, k, tm] * I[tm]['em']
        cnstr.add(s == C[k]['volume'])

In [10]:
# ,,,,,,,,,,,,,, Условие 5 ,,,,,,,,,,,,,,

def eval_cnstr5(cnstr, m, C, I):
    for k in m.K:
        b_k = C[k]['B']
        s = 0
        for tm in m.M:
            if k in I[tm]['Ctm']:
                for j in m.V:
                    s += m.q[b_k, j, k, tm] * I[tm]['em']
        cnstr.add(s == 0)

In [11]:
# ,,,,,,,,,,,,,, Условие 6 ,,,,,,,,,,,,,,

def eval_cnstr6(cnstr, m, C, Bandwidth):
    for tm in m.M:
        for k in m.K:
            a_k = C[k]['A']
            b_k = C[k]['B']
            for r in m.V - {a_k, b_k}:
                sum_in = 0
                sum_out = 0
                flag = False
                for j2 in m.V:
                    if Bandwidth.at[j2, r] != 0:
                        sum_in += m.q[j2, r, k, tm]
                        flag = True
                for j1 in m.V:
                    if Bandwidth.at[r, j1] != 0:
                        sum_out += m.q[r, j1, k, tm]
                        flag = True
                if flag:
                    cnstr.add((sum_in - sum_out) == 0)

In [12]:
# ,,,,,,,,,,,,,, Условие 7 ,,,,,,,,,,,,,,

def eval_cnstr7(cnstr, m, C, I):
    for i, j in product(m.V, m.V):
        for k in m.K:
            for tm in m.M - {max(m.M)}:
                if k in I[tm]['Ctm'] and k in I[tm + 1]['Ctm']:
                    cnstr.add(m.q[i, j, k, tm] == m.q[i, j, k, tm + 1])

In [13]:
# ,,,,,,,,,,,,,, Условие 8 ,,,,,,,,,,,,,,

def eval_cnstr8(cnstr, m, I):
    for i in m.V:
        for j in m.V:
            for tm in m.M:
                for k in m.K:
                    if not k in I[tm]['Ctm']: 
                        cnstr.add(m.q[i, j, k, tm] == 0)

In [14]:
# ,,,,,,,,,,,,,, Условие 9 ,,,,,,,,,,,,,,

def eval_cnstr9(cnstr, m):
    for tm in m.M:
        for k in m.K:
            for i in m.V:
                for j in m.V:
                    cnstr.add(m.q[i, j, k, tm] >= 0)

In [15]:
def get_result(result, m, time):
    correctness = ""
    answer = np.zeros([len(m.V), len(m.V), len(m.K), len(m.M)])
    if (result != None) and (result.solver.status == SolverStatus.ok) and (result.solver.termination_condition == TerminationCondition.optimal):
        correctness = "correct"
        for i, j, k, t in product(m.V, m.V, m.K, m.M):
            answer[i][j][k][t] = m.q[i, j, k, t]()
    else:
        correctness = "Model infeasible"        
        
    return (result, correctness, answer, time)

In [16]:
def evaluate_model(V, K, M, Contracts, Intervals, Bandwidth, BackFlows):
    
    start_time = time.time()
    
#     M_ind = []
#     for k in N:
#         M_ind.append(set(range(K[k]['time'] // e, T // e)))
    
    m = ConcreteModel()

    m.V = V
    m.K = K
    m.M = M

    m.q = Var(m.V, m.V, m.K, m.M, domain=NonNegativeIntegers)

    m.cnstr_01 = ConstraintList()
    zero_nconnect(m.cnstr_01, m, Bandwidth)

#     m.cnstr_02 = ConstraintList()
#     for k in m.K:
#         for t in M_ind[k]:
#             only_selftime(m.cnstr_02, m, k, t)

#     min_t = T;
#     for k in m.K:
#         min_t = min(K[k]['time'], min_t)

#     if not eval_cnstr0(m, min_t, K, Bandwidth, BackFlows):
#         end_time = time.time()
#         return get_result(None, m, end_time - start_time)

    m.cnstr_1 = ConstraintList()
    eval_cnstr1(m.cnstr_1, m, Intervals, Bandwidth, BackFlows)

    m.cnstr_2 = ConstraintList()
    eval_cnstr2(m.cnstr_2, m, Contracts, Intervals)
    
    m.cnstr_3 = ConstraintList()
    eval_cnstr3(m.cnstr_3, m, Contracts, Intervals)

    m.cnstr_4 = ConstraintList()
    eval_cnstr4(m.cnstr_4, m, Contracts, Intervals)
    
    m.cnstr_5 = ConstraintList()
    eval_cnstr5(m.cnstr_5, m, Contracts, Intervals)

    m.cnstr_6 = ConstraintList()
    eval_cnstr6(m.cnstr_6, m, Contracts, Bandwidth)
    
    m.cnstr_7 = ConstraintList()    
    eval_cnstr7(m.cnstr_7, m, Contracts, Intervals)
    
    m.cnstr_8 = ConstraintList()
    eval_cnstr8(m.cnstr_8, m, Intervals)
    
    m.cnstr_9 = ConstraintList()
    eval_cnstr9(m.cnstr_9, m)
    
    m.OBJ = Objective(expr = sum(m.q[i, j, k, t] for i, j, k, t in product(m.V, m.V, m.K, m.M)), sense=minimize)

    solver = SolverFactory('cbc')
    result = solver.solve(m)
    
    end_time = time.time()
    
    return get_result(result, m, end_time - start_time)